## Slicing
**(?)** Slicing in Julia returns a copy. What about in Python?  
**(R)** Slicing in Python returns a view.

In [23]:
v = ones(Int8, 7)

7-element Array{Int8,1}:
 1
 1
 1
 1
 1
 1
 1

In [12]:
w = v[4:end]

4-element Array{Int8,1}:
 1
 1
 1
 1

In [13]:
w[1:2] .= 127
w

4-element Array{Int8,1}:
 127
 127
   1
   1

In [14]:
v

7-element Array{Int8,1}:
 1
 1
 1
 1
 1
 1
 1

In [15]:
using PyCall

In [19]:
py"""
import numpy as np
vv = np.zeros(7, dtype=np.int8)
ww = vv[3:]
ww[:2] = -1
"""

In [20]:
py"ww"

4-element Array{Int8,1}:
 -1
 -1
  0
  0

In [21]:
py"vv"

7-element Array{Int8,1}:
  0
  0
  0
 -1
 -1
  0
  0

In [26]:
z = view(v, 4:end)

LoadError: syntax: missing last argument in "4:" range expression 

It seems that when using `view()` we cannot specify an index by `end`.

In [25]:
z = view(v, 4:7)

4-element view(::Array{Int8,1}, 4:7) with eltype Int8:
 1
 1
 1
 1

In [28]:
z[1:2] .= 17
v

7-element Array{Int8,1}:
  1
  1
  1
 17
 17
  1
  1

In [29]:
typeof(w), typeof(z)

(Array{Int8,1}, SubArray{Int8,1,Array{Int8,1},Tuple{UnitRange{Int64}},true})

In [30]:
z2 = @view v[4:end]

4-element view(::Array{Int8,1}, 4:7) with eltype Int8:
 17
 17
  1
  1

In [32]:
z2[1:2] .= -100
v

7-element Array{Int8,1}:
    1
    1
    1
 -100
 -100
    1
    1

## `reshape`, `ravel`
**(?)** Do they return a copy or a view?  
**(R)** Both Python and Julia returns views in this case.

- Python
  - `reshape` returns a view
  - `ravel`   returns a view (Thus, `ravel(order="C")` is equiv. to `reshape(-1, order="C")`)
- Julia
  - `reshape` returns a view
  - `vec` returns a view (`vec` is Julia's equivalent to Python's `np.ravel`)

In [40]:
py"""
shape = (3,5)
MM = np.arange(np.product(shape)).reshape(shape)
WW = MM.reshape(list(reversed(shape)))
"""

In [41]:
py"MM"

3×5 Array{Int64,2}:
  0   1   2   3   4
  5   6   7   8   9
 10  11  12  13  14

In [42]:
py"WW"

5×3 Array{Int64,2}:
  0   1   2
  3   4   5
  6   7   8
  9  10  11
 12  13  14

In [43]:
py"""
WW[0] = WW[0] * (-1)
"""

In [44]:
py"WW"

5×3 Array{Int64,2}:
  0  -1  -2
  3   4   5
  6   7   8
  9  10  11
 12  13  14

In [47]:
py"MM"

3×5 Array{Int64,2}:
  0  -1  -2   3   4
  5   6   7   8   9
 10  11  12  13  14

In [48]:
py"""
ZZ = np.ravel(MM)
"""

In [49]:
py"ZZ"

15-element Array{Int64,1}:
  0
 -1
 -2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14

In [50]:
py"""
ZZ *= -1
"""

In [51]:
py"MM"

3×5 Array{Int64,2}:
   0    1    2   -3   -4
  -5   -6   -7   -8   -9
 -10  -11  -12  -13  -14

Now let's see Julia.

In [62]:
M = [10i + j for i in 0:5, j in 1:4]

6×4 Array{Int64,2}:
  1   2   3   4
 11  12  13  14
 21  22  23  24
 31  32  33  34
 41  42  43  44
 51  52  53  54

In [63]:
W = reshape(M, 3, 8)

3×8 Array{Int64,2}:
  1  31   2  32   3  33   4  34
 11  41  12  42  13  43  14  44
 21  51  22  52  23  53  24  54

In [64]:
W[1, 1:3] .*= -1
W

3×8 Array{Int64,2}:
 -1  -31  -2  32   3  33   4  34
 11   41  12  42  13  43  14  44
 21   51  22  52  23  53  24  54

In [65]:
M

6×4 Array{Int64,2}:
  -1  -2   3   4
  11  12  13  14
  21  22  23  24
 -31  32  33  34
  41  42  43  44
  51  52  53  54

In [66]:
Z = vec(M)

24-element Array{Int64,1}:
  -1
  11
  21
 -31
  41
  51
  -2
  12
  22
  32
  42
  52
   3
  13
  23
  33
  43
  53
   4
  14
  24
  34
  44
  54

In [67]:
Z .*= -1
Z

24-element Array{Int64,1}:
   1
 -11
 -21
  31
 -41
 -51
   2
 -12
 -22
 -32
 -42
 -52
  -3
 -13
 -23
 -33
 -43
 -53
  -4
 -14
 -24
 -34
 -44
 -54

In [68]:
M

6×4 Array{Int64,2}:
   1    2   -3   -4
 -11  -12  -13  -14
 -21  -22  -23  -24
  31  -32  -33  -34
 -41  -42  -43  -44
 -51  -52  -53  -54